In [0]:
df = spark.read.table("main_prod.datascience_scratchpad.nyc_train_filtered")

display(df)

In [0]:
from pyspark.sql import functions as fn

# def get_min_max_xy(mer_seq):
#     min_x = min([x[0] for x in mer_seq])
#     min_y = min([x[1] for x in mer_seq])
#     max_x = max([x[0] for x in mer_seq])
#     max_y = max([x[1] for x in mer_seq])
#     return min_x, min_y, max_x, max_y



df = df.withColumn('min_x', fn.expr("array_min(merc_seq_filtered)[0]"))
df = df.withColumn('min_y', fn.expr("array_min(merc_seq_filtered)[1]"))
df = df.withColumn('max_x', fn.expr("array_max(merc_seq_filtered)[0]"))
df = df.withColumn('max_y', fn.expr("array_max(merc_seq_filtered)[1]"))
# df = df.withColumn('min_max_xy', fn.expr("array(min_x, min_y, max_x, max_y)"))
# df = df.drop('min_x', 'min_y', 'max_x', 'max_y')
display(df)

In [0]:
# get minimum value of min_x
df.select(fn.expr("min(min_x)")).show()
# get maximum value of max_x
df.select(fn.expr("max(max_x)")).show()
# get minimum value of min_y
df.select(fn.expr("min(min_y)")).show()
# get maximum value of max_y
df.select(fn.expr("max(max_y)")).show()

In [0]:
minx = df.select(fn.expr("min(min_x)")).collect()[0][0]
maxx = df.select(fn.expr("max(max_x)")).collect()[0][0]
miny = df.select(fn.expr("min(min_y)")).collect()[0][0]
maxy = df.select(fn.expr("max(max_y)")).collect()[0][0]
print(minx, maxx, miny, maxy)


In [0]:
minx = -8263433.7868088
maxx = -8230319.922972946
miny = 4988765.250473092
maxy = 4989832.581771174
offsetx = (k-(maxx-minx))/2
print(offsetx)

In [0]:
k = 100000
dx = (k - maxx - minx)/2
dy = (k - maxy - miny)/2
print(dx, dy)

In [0]:
from pyspark.sql.types import ArrayType, FloatType, DoubleType
def transform_traj(mer_seq):
    return [(x[0] + dx, x[1] + dy) for x in mer_seq]
transform_traj_udf = fn.udf(transform_traj, returnType=ArrayType(ArrayType(DoubleType())))
df = df.withColumn('merc_seq_transformed', transform_traj_udf('merc_seq_filtered'))
display(df)

In [0]:
df

In [0]:
import math
def meters2lonlat(merc_points):
    lon_lats = []
    for point in merc_points:
        x,y = point[0],point[1]
        semimajoraxis = 6378137.0
        lon = x / semimajoraxis / 0.017453292519943295
        t = math.exp(y / 3189068.5)
        lat = math.asin((t - 1) / (t + 1)) / 0.017453292519943295
        lon_lats.append([lon,lat])
    return lon_lats

In [0]:
transformed_lonlat_udf = fn.udf(meters2lonlat, returnType=ArrayType(ArrayType(DoubleType())))
df = df.withColumn('wgs_seq_transformed', transformed_lonlat_udf('merc_seq_transformed'))
display(df)

In [0]:
!pip install folium
import folium

from folium import plugins

def display_traj(lon_lat_list, ts_list = None):
    coordinates = [[x[1],x[0]] for x in lon_lat_list]
    # Initialize map
    m = folium.Map(location=coordinates[0], zoom_start=15)


    # for lat,lon in pip_list:
    #     folium.Marker(
    #         location=[lat, lon],
    #         popup="Pip Location",
    #         icon=folium.Icon(color='red')  # Color can be 'red', 'blue', 'green', 'purple', etc.
    #     ).add_to(m)

    folium.Marker(
        location=coordinates[0],
        popup="Start Location",
        icon=folium.Icon(color='green')  # Color can be 'red', 'blue', 'green', 'purple', etc.
    ).add_to(m)
    # Add markers
    for i, (lat, lon) in enumerate(coordinates[1:]):
        if ts_list is not None:
            folium.Marker([lat, lon], popup=ts_list[i+1]).add_to(m)
        else:
            folium.Marker([lat, lon], popup=i+1).add_to(m)

    # Draw arrows between points
    for i in range(len(coordinates) - 1):
        start = coordinates[i]
        end = coordinates[i + 1]

        # Draw the line
        line = folium.PolyLine([start, end], color="blue", weight=3, opacity=0.7).add_to(m)

        # Add directional arrow
        plugins.PolyLineTextPath(
            line,
            '➤',  # arrow symbol
            repeat=True,
            offset=7,
            attributes={'fill': 'blue', 'font-weight': 'bold', 'font-size': '16'}
            ).add_to(m)

    # Save and show the map
    m.save("map_with_arrows.html")
    return m


In [0]:
df.createOrReplaceTempView("temp_df")

In [0]:
query = """
select * from temp_df where traj_id rlike '16808692_*'
"""

df_fil = spark.sql(query)
display(df_fil)

In [0]:
df_pd = df_fil.toPandas()

In [0]:
df_pd.head()

In [0]:
traj_id = "16808692_2023-08-31"

display_traj(df_pd[df_pd['traj_id']==traj_id]['wgs_seq_transformed'].iloc[0])

In [0]:
traj_id = "16808692_2023-08-31"

display_traj(df_pd[df_pd['traj_id']==traj_id]['wgs_seq_filtered'].iloc[0])